**I. Insert, Update, Delete в psycopg2**

In [ ]:
# !pip install psycopg2-binary

*Импортируем библиотеку psycopg2*

In [ ]:
import psycopg2

*Создаём подключение к БД*

In [ ]:
conn = psycopg2.connect(host="rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net", 
                        port=6432,
                        database="demo", 
                        user="student",
                        password="student!"
                       )

*1) Последовательная вставка данных. Добавим оценки новых учеников.*

Сгенерируем данные для 4 новых учеников: Костя, Лена, Настя, Женя

In [ ]:
new_students = ['Костя', 'Лена', 'Настя', 'Женя']

Напишем функцию для генерации случайных оценок

In [ ]:
import random

def get_random_grade() -> int:
    """
    Function to generate random integers from 1 to 10
    
    return: integer from 1 to 10
    """
    
    grade = random.randint(1, 10)
    return grade

Проверим нашу функцию

In [ ]:
get_random_grade()

Напишем функцию генерации sql запроса на вставку данных

In [ ]:
def create_insert_sql(...) -> str:
    """
    Function to generate insert sql query
    
    return: insert sql query
    """
    
    sql = f'''
    INSERT INTO 
    ()
    VALUES
    ()
    ;
    '''

    return sql

Запустим генерацию оценок для новых учеников в цикле 

In [ ]:
with conn:
    with conn.cursor() as cur:
        for value in ...:
            cur.execute(sql)

*2) Обновление данных об оценках учеников.*

Обновим данные "старых" учеников (Маша и Петя):
- проставим в поле source значение "электронный журнал"
- проставим в поле updated_at значения из grade_dt

In [ ]:
sql = f'''
UPDATE
   SET ... = {}
 WHERE ...
'''

In [ ]:
with conn:
    with conn.cursor() as cur: 
        for value in ...:
            cur.execute(sql)

*Удаление неактуальных оценок.*

Удалим все строки для новых учеников

In [ ]:
sql = f'''
DELETE FROM ...
 WHERE ... 
'''

In [ ]:
with conn:
    with conn.cursor() as cur:      
        for value in ...:
            cur.execute(sql)

**II. Insert, Update, Delete в SQLAlchemy**

In [ ]:
# !pip uninstall sqlalchemy

In [ ]:
# !pip install sqlalchemy=='1.4.51'

In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [ ]:
from sqlalchemy import create_engine

In [ ]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase dwh"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [ ]:
engine = connection_yandex_cloud_demo(echo=True)

In [ ]:
from sqlalchemy.schema import MetaData

metadata_demo_db = MetaData(schema='public')
metadata_demo_db.reflect(bind=engine);

Загружаем данные о таблицах

In [ ]:
for table_name in metadata_demo_db.tables:
    print(table_name)

In [ ]:
from sqlalchemy import Table

student_grades = Table('student_grades', metadata_demo_db, autoload=True)

Вставка значений INSERT

In [ ]:
from sqlalchemy import insert

ins = student_grades.insert().values(
    name = 'test_name',
    subject = 'test_subject'
)

In [ ]:
print(ins)

In [ ]:
ins.compile().params

In [ ]:
engine.execute(ins)

Вставка нескольких записей

In [ ]:
engine.execute(insert(student_grades), [
        {
            "name": "-1", 
            "subject": "subject_1"
        },
        {
            "name": "-2", 
            "subject": "subject_2"
        },
        {
            "name": "-3", 
            "subject": "subject_3"
        },
    ])

In [ ]:
engine.execute(f'''
INSERT INTO public.student_grades
("name", "subject")
VALUES
(-1, 'subject_1'),
(-2, 'subject_2'),
(-3, 'subject_3')
;
''')

Обновление записей UPDATE

In [ ]:
from sqlalchemy import update

sql_query = update(student_grades).where(
    student_grades.c.id_student == 1
).values(
    grade = 7
)

print(sql_query)

In [ ]:
engine.execute(sql_query)

Удаление записей DELETE

In [ ]:
from sqlalchemy import delete

sql_query = delete(student_grades).where(
    student_grades.c.id_student == ???
)

print(sql_query)

In [ ]:
engine_demo.execute(sql_query)

**III. Insert в Pandas**

*Импортируем библиотеки*

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

*Загрузим данные в БД*

In [ ]:
d = {'name': ['Гриша', 'Слава'], 
     'grade': [1, 4]
    }
df = pd.DataFrame(data=d)
df

In [ ]:
df.to_sql(name='student_grades'
         ,con=engine
         ,schema='public'
         ,if_exists='append'
         ,index=False)